In [239]:
! pip install numpy pandas seaborn matplotlib scikit-learn xgboost

In [294]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV


from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, classification_report, precision_score, recall_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


import warnings
warnings.filterwarnings('ignore')

# job = pd.read_csv('/Users/macbookair/Desktop/project/Resume_Analyser/resume-job-matching/data/Preprocessed/Cleaned_job.csv')
resume = pd.read_csv('/Users/macbookair/Desktop/project/Resume_Analyser/resume-job-matching/data/Preprocessed/Cleaned_resume.csv')

In [295]:
pd.set_option('max_colwidth', 800)

In [296]:
resume.head(2)

,Category,cleaned_resume
0,data science,programming language python pandas numpy scipy scikitlearn matplotlib sql java javascriptjquery machine learn regression svm nave baye knn random forest decision tree boost technique cluster analysis word embed sentiment analysis natural language process dimensionality reduction topic model lda nmf pca neural net database visualization mysql sqlserver cassandra hbase elasticsearch d3js dcjs plotly kibana matplotlib ggplot tableau other regular expression html css angular 6 logstash kafka python flask git docker computer vision open cv understand deep learningeducation detail datum science assurance associate datum science assurance associate ernst young llp skill detail javascript exprience 24 month jquery exprience 24 month python exprience 24 monthscompany detail company ernst young ...
1,data science,education detail may 2013 may 2017 uitrgpv data scientist data scientist matelabs skill detail python exprience less 1 year month statsmodels exprience 12 month aw exprience less 1 year month machine learn exprience less 1 year month sklearn exprience less 1 year month scipy exprience less 1 year month keras exprience less 1 year monthscompany detail company matelab description ml platform business professional dummy enthusiast 60a koramangala 5th block achievementstask behind sukh sagar bengaluru india develop deploy auto preprocessing step machine learn mainly miss value treatment outli detection encode scale feature selection dimensionality reduction deploy automate classification regression model linkedincominadityarathore b4600b146 reasearch deploy time series forecasting model ar...


In [297]:
resume.tail(2)

,Category,cleaned_resume
3444,automobile,customer relation specialist summary obtain position company offer chance utilize call center customer service good ability well opportunity advancement competitive salary highly personable call center representative customer service collection call center operation accomplishment reach monthly sale goal several occasion customer relation specialist january 2015 current company name city state provide honda client resource register concern complaint request assistance outline owner manual ensure good possible outcome respond client question concern provide solution whenever possible professional helpful knowledgeable timely manner accurately capture document client information support voice client executive reporting program improvement customer service representative april 2014 octobe...
3445,automobile,senior special investigator summary work within successful special investigation unit work effectively other work swear police officer 10 year employ sr special investigator manager within insurance industry 30 year work insurance company special investigation unit combat fraud investigate questionable claim claim determine fraudulent element fraud file numerous case states doi fraud bureau appropriate county district attorney office also conduct numerous training session insurance industry recognize red flag insurance fraud highlight insurance fraud expertise database management team player strong interpersonal communication interview technique property claim decisive worker compensation claim critical thinker report write accomplishment earn certify insurance fraud investigator cifi ...


In [298]:
resume.isnull().sum()

Category          0
cleaned_resume    1
dtype: int64

In [299]:
resume['cleaned_resume'] = resume['cleaned_resume'].fillna("")

## Model Building

In [300]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(resume['Category'])
resume['Category'] = le.transform(resume['Category'])

In [301]:
resume.Category.unique()

array([17, 31,  1,  4, 44, 34, 40, 29, 14, 33, 11, 41,  5, 23, 36, 39, 20,
       35, 37, 18, 28, 25, 22,  9, 43,  2, 16, 21, 13, 30, 38,  7,  8,  0,
       32, 15, 19, 26, 27, 12,  3, 10, 42, 24,  6])

In [302]:
resume.describe()

,Category
count,3446.000000
mean,21.171503
std,13.257237
min,0.000000
25%,9.000000
50%,21.000000
75%,32.000000
max,44.000000


In [303]:
# Create a dictionary to map category ID (label) to category name
category_dict = {label: class_name for label, class_name in enumerate(le.classes_)}

# Print the dictionary
print(category_dict)

{0: 'accountant', 1: 'advocate', 2: 'agriculture', 3: 'apparel', 4: 'arts', 5: 'automation testing', 6: 'automobile', 7: 'aviation', 8: 'banking', 9: 'blockchain', 10: 'bpo', 11: 'business analyst', 12: 'business-development', 13: 'chef', 14: 'civil engineer', 15: 'construction', 16: 'consultant', 17: 'data science', 18: 'database', 19: 'designer', 20: 'devops engineer', 21: 'digital-media', 22: 'dotnet developer', 23: 'electrical engineering', 24: 'engineering', 25: 'etl developer', 26: 'finance', 27: 'fitness', 28: 'hadoop', 29: 'health and fitness', 30: 'healthcare', 31: 'hr', 32: 'information-technology', 33: 'java developer', 34: 'mechanical engineer', 35: 'network security engineer', 36: 'operations manager', 37: 'pmo', 38: 'public-relations', 39: 'python developer', 40: 'sales', 41: 'sap developer', 42: 'teacher', 43: 'testing', 44: 'web designing'}


#### TF-IDF Vectorization and Train - Test Split

In [304]:
tfidf = TfidfVectorizer(stop_words='english')

tfidf.fit(resume['cleaned_resume'])
resumeTextTFIDF  = tfidf.transform(resume['cleaned_resume'])

In [305]:
resumeTextTFIDF

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 886395 stored elements and shape (3446, 52284)>

In [306]:
X_train, X_test, y_train, y_test = train_test_split(resumeTextTFIDF, resume['Category'], test_size=0.2, random_state=42)

In [307]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2756, 52284) (690, 52284) (2756,) (690,)


In [308]:
y_train, y_test

(2984     3
 829     22
 3076     1
 1878     7
 2069     0
         ..
 1095     4
 1130    40
 1294    16
 860      9
 3174    10
 Name: Category, Length: 2756, dtype: int64,
 443     41
 51      31
 794     25
 1572    13
 1437    21
         ..
 2490    15
 3417     6
 73      31
 48      31
 1728    38
 Name: Category, Length: 690, dtype: int64)

### Classification Models

##### Logistic Regression

In [309]:
model_lgr = LogisticRegression()
model_lgr.fit(X_train, y_train)

y_pred = model_lgr.predict(X_test)

accuracy_lgr = accuracy_score(y_test, y_pred)

print("Logistic Regression Accuracy:", accuracy_lgr)

Logistic Regression Accuracy: 0.7782608695652173


##### KNN Classifier

In [310]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)

y_pred_knn = model_knn.predict(X_test)
accuracy_knn = accuracy_score(y_test, y_pred_knn)

print("KNN Classifier Accuracy:", accuracy_knn)

KNN Classifier Accuracy: 0.6710144927536232


##### Decision Tree Classifier

In [311]:
model_dtc = DecisionTreeClassifier()
model_dtc.fit(X_train, y_train)

y_pred_dtc = model_dtc.predict(X_test)
accuracy_dtc = accuracy_score(y_test, y_pred_dtc)

print("Decision Tree Classifier Accuracy:", accuracy_dtc)

Decision Tree Classifier Accuracy: 0.7188405797101449


##### Random Forest Classifier

In [312]:
model_rfc = RandomForestClassifier(n_estimators=100, random_state=42)
model_rfc.fit(X_train, y_train)

y_pred_rfc = model_rfc.predict(X_test)
accuracy_rfc = accuracy_score(y_test, y_pred_rfc)

print("Random Forest Classifier Accuracy:", accuracy_rfc)

Random Forest Classifier Accuracy: 0.7710144927536232


##### Support Vectore Machine (SVM) Classifier

In [313]:
model_svm = SVC()
model_svm.fit(X_train, y_train)

y_pred_svm = model_svm.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print("SVM Classifier Accuracy:", accuracy_svm)

SVM Classifier Accuracy: 0.7405797101449275


##### Bagging Classifier

In [279]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

model_bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42)

model_bagging.fit(X_train, y_train)

y_pred_bagging = model_bagging.predict(X_test)

accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f"Bagging Classifier Accuracy: {accuracy_bagging}")

Bagging Classifier Accuracy: 0.8304347826086956


##### Gradient Boosting Classifier

In [278]:
model_gbc = GradientBoostingClassifier()
model_gbc.fit(X_train, y_train)

y_pred_gbc = model_gbc.predict(X_test)
accuracy_gbc = accuracy_score(y_test, y_pred_gbc)

print("Gradient Boosting Classifier Accuracy:", accuracy_gbc)

Gradient Boosting Classifier Accuracy: 0.8463768115942029


##### Naive Bayes Classifier

In [314]:
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

y_pred_nb = model_nb.predict(X_test)
accuracy_nb = accuracy_score(y_test, y_pred_nb)

print("Naive Bayes Classifier Accuracy:", accuracy_nb)

Naive Bayes Classifier Accuracy: 0.5594202898550724
